In [1]:
# Gereken Kütüphaneler
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
import gc
import joblib

## 1. Veriyi Feature Engineering Eklenmiş Haliyle Hazırlama

In [2]:
def get_data_with_features():
    print("Loading data with features. Please wait this may take a while...")
    df = pd.read_csv('../pubg_predictor/data/train_V2.csv')
    df = df.dropna(subset=['winPlacePerc'])

    df['totalDistance'] = df['walkDistance'] + df['rideDistance'] + df['swimDistance']
    df['healthItems'] = df['heals'] + df['boosts']
    df['headshotRate'] = df['headshotKills'] / df['kills']
    df['headshotRate'] = df['headshotRate'].fillna(0)
    df['teamwork'] = df['revives'] + df['assists']
    
    
    df['matchMeanKills'] = df.groupby('matchId')['kills'].transform('mean')
    df['matchMeanDamage'] = df.groupby('matchId')['damageDealt'].transform('mean')
    df['killsRel'] = df['kills'] / df['matchMeanKills']
    df['damageRel'] = df['damageRel'] = df['damageDealt'] / df['matchMeanDamage']
    df['killsRel'] = df['killsRel'].fillna(0)
    df['damageRel'] = df['damageRel'].fillna(0)
    
    
    df = df.drop(columns=['Id', 'groupId', 'matchId'])
    
    # One-Hot Encoding
    df = pd.get_dummies(df, columns=['matchType'])
    
    return df

df = get_data_with_features()

Loading data with features. Please wait this may take a while...


Kod tekrarı olmaması açısından bir önceki kısımda yaptığımız şeyleri fonksiyon olarak tanımladık.

## 2. Optimizasyon

Veri setimiz 4 milyon satırdan oluştuğu için tamamını kullanmak çok zaman alır, bu nedenle %10 kadarıyla çalışıp, en iyi ayarı bulduktan sonra geneline uygulamak daha verimli olacaktır.

In [4]:
sample_df = df.sample(frac=0.1, random_state=42)
print(f"Sampled DataFrame shape: {sample_df.shape}")

x_sample = sample_df.drop(columns=['winPlacePerc'])
y_sample = sample_df['winPlacePerc']

gc.collect() # Bellek Temizleme

Sampled DataFrame shape: (444696, 49)


770

## 3. Hiperparametre

In [5]:
param_grid = {
    'num_leaves': [31, 50, 70],          # Ağacın karmaşıklığı
    'learning_rate': [0.01, 0.05, 0.1],  # Öğrenme hızı
    'n_estimators': [100, 500],          # Ağaç sayısı
    'min_child_samples': [20, 50],       # Overfitting engellemek için
    'subsample': [0.7, 0.9],             # Her ağaçta verinin yüzde kaçı kullanılsın
    'colsample_bytree': [0.7, 0.9]       # Her ağaçta feature'ların yüzde kaçı kullanılsın
}

## 4. Random Search

In [6]:
lgbm = lgb.LGBMRegressor(random_state=42, n_jobs=-1)

random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_grid,
    n_iter=10,             # 10 farklı kombinasyon deneyecek (Sayısı artarsa süre uzar)
    scoring='neg_mean_absolute_error',
    cv=3,                  # 3 katlı çapraz doğrulama
    verbose=2,
    random_state=42,
    n_jobs=-1
)
print("Starting Randomized Search CV. This may take a while...")
random_search.fit(x_sample, y_sample)

print("-" * 40)
print(f"🏆 BEST PARAMETERS: {random_search.best_params_}")
print(f"🏆 NEW BEST SCORE: {-random_search.best_score_}")
print("-" * 40)

Starting Randomized Search CV. This may take a while...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016200 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4235
[LightGBM] [Info] Number of data points in the train set: 444696, number of used features: 48
[LightGBM] [Info] Start training from score 0.471443
----------------------------------------
🏆 BEST PARAMETERS: {'subsample': 0.7, 'num_leaves': 50, 'n_estimators': 500, 'min_child_samples': 20, 'learning_rate': 0.1, 'colsample_bytree': 0.7}
🏆 NEW BEST SCORE: 0.05608691059444829
----------------------------------------


## 5. Optimize Model ile Tüm Veriyi Eğitme

In [7]:

print("Training final model with best parameters on full dataset...")
best_params = random_search.best_params_

X = df.drop(columns=['winPlacePerc'])
y = df['winPlacePerc']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

final_model = lgb.LGBMRegressor(**best_params, random_state=42, n_jobs=-1)
final_model.fit(X_train, y_train)

final_preds = final_model.predict(X_val)
final_mae = mean_absolute_error(y_val, final_preds)

print(f"🚀 OPTIMIZED FINAL MAE SCORE: {final_mae:.5f}")



Training final model with best parameters on full dataset...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4228
[LightGBM] [Info] Number of data points in the train set: 3557572, number of used features: 48
[LightGBM] [Info] Start training from score 0.472937
🚀 OPTIMIZED FINAL MAE SCORE: 0.05522


## 6. Modeli Kaydet

In [8]:
import os
if not os.path.exists('../pubg_predictor/models'):
    os.makedirs('../pubg_predictor/models')

joblib.dump(final_model, '../pubg_predictor/models/pubg_model.pkl')
print("Model '../pubg_predictor/models/pubg_model.pkl' olarak kaydedildi!")

Model '../pubg_predictor/models/pubg_model.pkl' olarak kaydedildi!
